In [1]:
import requests
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
from time import sleep
from random import randint

# API Camp to Camp - 

In [2]:
#Export data from C2C API

url = 'https://api.camptocamp.org/outings'
france = '&a=14274'
req_activities = {'?act=skitouring':310,'?act=rock_climbing':330,'?act=mountain_climbing':100,'?act=snow_ice_mixed':100,'?act=ice_climbing':100,'?act=hiking':100}

request_num = 0
result=[]

for k,v in req_activities.items():
    for i in range(0,v):
        if i == 0 : 
            api_path = url + k + france
        else:
            api_path = url + k + france + '&offset=' + str(i*30)            
        r = requests.get(api_path)
        result.append(r.json())
        request_num += 1
        sleep(randint(1,3))
        
print("Nombre total de reqûetes :" + str(request_num))

Nombre total de reqûetes :1040


# # Build dataframe & export to csv - 

In [3]:
# Flatten json file - 
d = pd.DataFrame()

for k in range(len(result)):    
    d=d.append(json_normalize(result[k]['documents'],max_level=1))

d.reset_index(inplace=True, drop=True)

# Select relevant columns -  
cols = ['document_id','activities','date_start','date_end','author.name','author.user_id']
d=d[cols]

# Print first rows
d.head()

,document_id,activities,date_start,date_end,author.name,author.user_id
0,1196222,[skitouring],2020-03-23,2020-03-23,christian73,1071965
1,1195244,"[skitouring, snow_ice_mixed, mountain_climbing...",2020-03-17,2020-03-17,dormeur10,800213
2,1195133,[skitouring],2020-03-17,2020-03-17,christian73,1071965
3,1195132,[skitouring],2020-03-17,2020-03-17,janic,578085
4,1195116,"[skitouring, snow_ice_mixed, paragliding]",2020-03-17,2020-03-17,Paulo,991164


In [4]:
# Clean columns -

# Clean activity - 
def dummies_activity(x,a):
    if a in x:
        return True
    else:
        return False

activities=['skitouring', 'rock_climbing','mountain_climbing','ice_climbing','hiking','snow_ice_mixed']

for activity in activities:
    d[activity]=d.activities.apply(lambda x: dummies_activity(x,activity))

d.drop('activities', inplace=True, axis=1)

# Drop_duplicates (caused by multi-activities outings exported several times during requests)
d.drop_duplicates(inplace=True)

# Convert to datetime
col_to_date=['date_end','date_start']
d[col_to_date]=d[col_to_date].apply(lambda x: pd.to_datetime(x))

# Convert to integer
col_to_int=['document_id','author.user_id']
d[col_to_int]=d[col_to_int].astype(int)

# Export to csv -
d.to_csv('outings.csv',index=False)

# CHECK - 
d.head()

,document_id,date_start,date_end,author.name,author.user_id,skitouring,rock_climbing,mountain_climbing,ice_climbing,hiking,snow_ice_mixed
0,1196222,2020-03-23,2020-03-23,christian73,1071965,True,False,False,False,False,False
1,1195244,2020-03-17,2020-03-17,dormeur10,800213,True,False,True,True,False,True
2,1195133,2020-03-17,2020-03-17,christian73,1071965,True,False,False,False,False,False
3,1195132,2020-03-17,2020-03-17,janic,578085,True,False,False,False,False,False
4,1195116,2020-03-17,2020-03-17,Paulo,991164,True,False,False,False,False,True


In [ ]:
df = pd.read_csv('outings.csv')
df.head()

In [ ]:
#>>>>>Trouver une question plus sympa - 
#>>EDA on number of outings - 

#Etude des sorties - Année 2018 et 2019 
date_start = datetime(year=2017,month=12,day=31)
date_end = datetime(year=2019,month=12,day=31)
df_outings = d[(d['date_start']>date_start) & (d['date_end']<=date_end)] 

#>>>Plot des séries temporelles à revoir - 
# #>>Voir comment faire le graph temporel en plus propose
# Quand pratiquait à quelle période de l'année?
#Biais des vacances dans le nombre de sorties - en terme de disponibilité - intéressant sur rock_climbing - sur représentation de la période - 
#Quand peut-on pratiquait?

# df_outings_daily = df_outings.groupby(by=[pd.Grouper(key='date_start',freq='D'),'activities']).count()
# df_outings_daily.reset_index(inplace=True)
# df_outings_daily.head()

# # test = df_outings.groupby(by=[pd.Grouper(key='date_start',freq='M'),'activities'])['document_id','author.user_id'].count()
# # test.reset_index(inplace=True)
# # test.head(15)
# # test.dropna(subset='document_id',inplace=True,axis=0)

# sns.set()
# sns.lineplot(x='date_start',y='document_id',hue='activities',data=df_outings_daily)
# plt.show()
         
        
#Heatmap des pratiques par mois
#>>>Streamliner la construction de la heatmap? - 

df_outings['month']=df_outings.date_start.dt.month
# df_outings['month']=df_outings.date_start.apply(lambda x: x.strftime('%B'))

df_outings_by_act_month_wide=df_outings.groupby('month',as_index=False)[activities].sum()
df_outings_by_act_month = pd.melt(df_outings_by_act_month_wide,id_vars=['month'],var_name='activities',value_vars=activities,value_name='number_outings')
df_outings_by_act_month.set_index(['activities','month'], inplace=True)
df_outings_by_act = df_outings_by_act_month.groupby('activities').sum()
df_outings_by_act.head()

df_outings_by_act_month_pcts = df_outings_by_act_month.div(df_outings_by_act,level='activities',axis=0)*100
df_outings_by_act_month_pcts.head()

#Check - Somme des activites à 100 - OK
# df_outings_by_act_month_pcts.groupby('activities').sum()

df_heatmap = pd.pivot_table(df_outings_by_act_month_pcts,index='activities',columns='month',values='number_outings')
df_heatmap.columns=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(df_heatmap, annot=True, linewidths=.1, ax=ax,fmt=".0f",cmap='Blues')
plt.title("Nature des pratiques de montagne au cours de l'année")
plt.show()         

#INSIGHT - When to is it best to practice? e.g. when most people do - hypothèse contraintes de conditions - when you can people practices - 
#INSIGHT -Pose la question de la polyvalence des pratiquants - 
#IN - Comportement moyen des ytilisateurs - est-ce que ça cache différents types de pratiques ou de la polyvalence?